## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Algoritmo Genético para asignación de quirófanos

#### Autor: Jesús García Armario

In [1]:
# Imports necesarios
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import sys
sys.path.append('../')
from Heuristicas.Utils import Quirofano, ActoQuirurgico
import random
import math

In [2]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]
# Creamos un set de actos quirúrgicos
actos_pendientes = list()
i = 0
for elemento in df.itertuples():
    actos_pendientes.append(ActoQuirurgico(i, elemento[3], elemento[1], elemento[2]))
    i += 1

In [3]:
# Definimos un cromosoma como una lista de tamaño N
# donde N es el número de slots temporales disponibles
# El ID de la operación ocupará la posición del slot temporal o 0 si no se ha asignado
# El cromosoma se codifica como una lista de enteros
# Con separadores representados por caracteres especiales, siendo 'A' el separador de quirófanos
# y 'B' el separador de días
def cromosomaAleatorio(tiempos, quirofanos, dias, actos_pendientes, ventana):
    cromosoma = []
    for i in range(dias):
        for j in range(quirofanos):
            tDisponibles = tiempos
            while tDisponibles > 0:
                # Seleccionamos un elemento al azar de actos_pendientes
                acto = random.choice(actos_pendientes)
                T = math.ceil(acto.getDuracion()/ventana)
                # Lo añadimos al cromosoma T veces
                for l in range(T):
                    cromosoma.append(acto.getId())
                # Añadimos un separador de intervenciones
                cromosoma.append(0)
                # Avanzamos el puntero de tiempos
                actos_pendientes.remove(acto)
                tDisponibles -= (T + 1)
            cromosoma.append('A')
        cromosoma.append('B')
    return cromosoma

In [4]:
# Definimos al individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("cromosoma", cromosomaAleatorio, 16, 3, 5, actos_pendientes, 30)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.cromosoma)
toolbox.register("poblacion", tools.initRepeat, list, toolbox.individual)

In [5]:
# Definimos la función de fitness
def evaluar(individual):
    # Definimos los parámetros de la evaluación
    # Número de quirófanos
    nQuirofanos = 3
    # Número de días
    nDias = 5
    # Número de slots temporales
    nSlots = 16
    # Ventana de tiempo
    ventana = 30
    # Definimos la lista de quirófanos
    quirofanos = []
    for i in range(nQuirofanos):
        quirofanos.append(Quirofano(i, nDias, nSlots))
    # Definimos la lista de actos quirúrgicos
    actos = []
    for i in range(len(individual)):
        if individual[i] != 'A' and individual[i] != 'B':
            actos.append(actos_pendientes[individual[i]])
    # Asignamos los actos quirúrgicos a los quirófanos
    for i in range(len(individual)):
        if individual[i] != 'A' and individual[i] != 'B':
            # Calculamos el día y el slot temporal
            dia = i // nSlots
            slot = i % nSlots
            # Asignamos el acto quirúrgico al quirófano correspondiente
            quirofanos[dia].addActo(actos_pendientes[individual[i]])
    # Calculamos el fitness
    fitness = 0
    for quirofano in quirofanos:
        for acto in quirofano.getActos():
            fitness += acto.getPrioridad()
    return fitness,

In [6]:
def cruce(ind1, ind2):
    # Obtenemos las posiciónes de los separadores
    posA1 = ind1.index('A')
    posA2 = ind2.index('A')
    posB1 = ind1.index('B')
    posB2 = ind2.index('B')
    # Intercambiamos los cromosomas
    ind1[posA1:posB1], ind2[posA2:posB2] = ind2[posA2:posB2], ind1[posA1:posB1]
    return ind1, ind2

In [7]:
toolbox.register("evaluate", evaluar)
toolbox.register("mate", cruce)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [8]:
# Definimos los parámetros del algoritmo
# Tamaño de la población
tPoblacion = 100
# Probabilidad de cruce
pCruce = 0.8
# Probabilidad de mutación
pMutacion = 0.2
# Número de generaciones
nGeneraciones = 100
# Creamos la población inicial
poblacion = toolbox.poblacion(n=tPoblacion)

IndexError: Cannot choose from an empty sequence